In [38]:
using JuMP, Gurobi
# Pkg.add("Combinatorics")
using Combinatorics


In [39]:
caldata_demand = Dict("A" => Dict("B" => 38000, "D"=>7000, "E"=> 5000),
    "B" => Dict( "D"=>19000, "C"=> 1000, "A"=>42000, "E"=>7000),
    "D" => Dict("C"=>1000, "A"=>12000),
    "C" => Dict("A"=>1000)
);

# demands["AB"] = n units to send FROM A TO B
demands = Dict()
for hub in keys(caldata_demand)
    for other_hub in keys(caldata_demand[hub])
        d = caldata_demand[hub][other_hub]
        demands[hub*other_hub] = d
    end
end

commodity_keys = collect(keys(demands))
demand_vector = zeros(1:length(demands))
commodities = Dict()
for (i, key) in enumerate(commodity_keys)
    commodities[i] = key
    demand_vector[i] = demands[key]
end

# commodity_keys = [1,2,3,...10]
# demand vector = [38000, 7000, ...]
# commodities = {1: "AB", ... , 10:"AE"}

n_commodities = length(commodities)
n_commodities

10

In [40]:
demands

Dict{Any,Any} with 10 entries:
  "BE" => 7000
  "AE" => 5000
  "DC" => 1000
  "CA" => 1000
  "AD" => 7000
  "BC" => 1000
  "BD" => 19000
  "DA" => 12000
  "BA" => 42000
  "AB" => 38000

In [41]:
adm_cost_per_unit_data = 2;
bbdx_cost_per_unit_data = 4; # plus 2 for each ADM
bidirectional_arc_capacity = 24000; 
adm_capacity = 48000; # because each ADM is attached to two arcs
adm_cost = 1000000;

In [42]:
function flowcost(hub1hub2)
    if hub1hub2 in ["BD", "DB", "DE", "ED", "EB", "BE", "BA", "AB", "AC", "CA", "CE", "EC"]
        return 1
    else
        return 0
    end
end;

In [43]:
function hasbbdx(hub)
    return ! (hub == "C" || hub=="F" || hub =="G" || hub=="I" || hub=="M")
end;

In [44]:
type_0_nodes = ["A", "B", "C", "D", "E"];
cycles = Dict("ABEC"=>4, "BDE"=>2);

In [45]:
# n_type_1_nodes = 63
# n_type_0_nodes = 14

node_indices_to_descriptions = Dict()
index = 1
# for t0node in type_0_nodes
#     node_indices_to_descriptions[index] = "hub_" * t0node
#     index += 1
# end

for cycle in keys(cycles)
    for hub in cycle
        for ring in 1:cycles[cycle]
            node_indices_to_descriptions[index] = "hub_" * hub * "_cycle_" * cycle * "_ring_" * string(ring)
            index += 1
        end
    end
end

In [46]:
hubs_to_cycles = Dict()
for cycle in keys(cycles)
    for hub in cycle
        if string(hub) in keys(hubs_to_cycles)
            push!(hubs_to_cycles[string(hub)], cycle)
        else
            hubs_to_cycles[string(hub)] = [cycle]
        end
    end
end
hubs_to_cycles

Dict{Any,Any} with 5 entries:
  "B" => String["ABEC", "BDE"]
  "A" => String["ABEC"]
  "C" => String["ABEC"]
  "D" => String["BDE"]
  "E" => String["ABEC", "BDE"]

In [47]:
num_nodes = length(node_indices_to_descriptions)
n_type_1_nodes = num_nodes

22

In [48]:
node_descriptions_to_indices = Dict()
for k in keys(node_indices_to_descriptions)
   node_descriptions_to_indices[node_indices_to_descriptions[k]] = k 
end

In [49]:
# node_descriptions_to_indices

### Arcs

In [50]:
arc_indices_to_descriptions = Dict()
arc_descriptions_to_indices = Dict()

ind = 1
# type 0 and type 1 arcs: from each hub to ring, and from each ring to the hub, respectively
for hub in type_0_nodes
    for cycle in hubs_to_cycles[hub]
        for ring in 1:cycles[cycle]
            # from city to ring
            arc_indices_to_descriptions[ind] = "hub_" * hub * "_to_hub_" * hub * "_cycle_" *
                                        cycle * "_ring_" * string(ring)
            ind += 1
            # from ring to city
            arc_indices_to_descriptions[ind] = "hub_" * hub * "_cycle_" * cycle * "_ring_" * 
                                        string(ring) * "_to_hub_" * hub
            ind += 1
        end
    end
end


println("after type 0 and 1: "*string(ind))
# type 2 arcs: one hub to the next along a ring (bidirectional, not symmetric)
for cycle in keys(cycles)
    for ring in 1:cycles[cycle]
        for i in 1:(length(cycle)-1)
            from_hub = cycle[i]
            to_hub = cycle[i+1]
            arc_indices_to_descriptions[ind] = "hub_" * string(from_hub) * "_cycle_" * cycle * "_ring_" * 
                    string(ring) * "_to_hub_" * to_hub * "_cycle_" * cycle * "_ring_" * string(ring)
            ind += 1
            arc_indices_to_descriptions[ind] = "hub_" * to_hub * "_cycle_" * cycle * "_ring_" * string(ring) * 
                                    "_to_hub_" * from_hub * "_cycle_" * cycle * "_ring_" * string(ring)
            ind += 1
        end
        arc_indices_to_descriptions[ind] = "hub_" * cycle[1] * "_cycle_" * cycle * "_ring_" * string(ring) * 
                            "_to_hub_" * cycle[end] * "_cycle_" * cycle * "_ring_" * string(ring)
        ind += 1
        
        arc_indices_to_descriptions[ind] = "hub_" * cycle[end] * "_cycle_" * cycle * "_ring_" * string(ring) * 
                        "_to_hub_" * cycle[1] * "_cycle_" * cycle * "_ring_" * string(ring)
        ind += 1
    end
end


println("after type 2: "*string(ind))

# type 3 arcs: These go from one ring to another at a hub that has a BBDX. 
# Any commodity may flow on these arcs, but only if an ADM is installed at the hub on both rings. 
for hub in type_0_nodes
    ### TODO: FIX THIS! WRONG LOGIC?? ###
    
    
    if !hasbbdx(hub)
        continue ### change this to a constraint of zero?
    end
    
    cycles_at_hub = hubs_to_cycles[hub]
    for cycle_num in 1:length(cycles_at_hub)
        # in-cycle transfers: across rings
        for pair in collect(combinations(collect(1:cycles[cycles_at_hub[cycle_num]]), 2))
            ring1, ring2 = pair
#             println(hub * ": " * string(cycle_num) * ": "* string(ring1) * string(ring2))
            arc_indices_to_descriptions[ind] = "hub_"*hub*"_cycle_"*cycles_at_hub[cycle_num]*"_ring_"*string(ring1) *
                                                        "_to_hub_"*hub*"_cycle_"*cycles_at_hub[cycle_num]*"_ring_"*string(ring2)
            ind += 1
            arc_indices_to_descriptions[ind] = "hub_"*hub*"_cycle_"*cycles_at_hub[cycle_num]*"_ring_"*string(ring2) *
                                                        "_to_hub_"*hub*"_cycle_"*cycles_at_hub[cycle_num]*"_ring_"*string(ring1)
            ind += 1
        end

        # out-of-cycle transfers
        for other_cycle_num in append!(collect(1:cycle_num-1), collect(cycle_num+1:length(cycles_at_hub)))
            # for each ring in THIS cycle, make a link to the other ring in other cycles
            for ring in 1:cycles[cycles_at_hub[cycle_num]]
                for other_ring in 1:cycles[cycles_at_hub[other_cycle_num]]
                    if cycle_num == other_cycle_num
                        print(cycle_num)
                    end

                    arc_indices_to_descriptions[ind] = "hub_"*hub*"_cycle_"*cycles_at_hub[cycle_num]*"_ring_"*
                                                        string(ring) * "_to_hub_"*hub*"_cycle_"*
                                                        cycles_at_hub[other_cycle_num]*"_ring_"*string(other_ring)
                    ind += 1
                end
            end
        end
    end
end
   


for k in keys(arc_indices_to_descriptions)
   arc_descriptions_to_indices[arc_indices_to_descriptions[k]] = k 
end

num_arcs = length(arc_indices_to_descriptions)

after type 0 and 1: 45
after type 2: 89


162

In [51]:
caldata = Model(solver=GurobiSolver())
BIGM = 24000 # the max arc capacity

24000

### Decision Variables
- how much data to send on each arc
- where to locate the ADMs

In [52]:
@variable(caldata, arcflows[1:num_arcs, 1:n_commodities] >= 0)

162×10 Array{JuMP.Variable,2}:
 arcflows[1,1]    arcflows[1,2]    …  arcflows[1,9]    arcflows[1,10]  
 arcflows[2,1]    arcflows[2,2]       arcflows[2,9]    arcflows[2,10]  
 arcflows[3,1]    arcflows[3,2]       arcflows[3,9]    arcflows[3,10]  
 arcflows[4,1]    arcflows[4,2]       arcflows[4,9]    arcflows[4,10]  
 arcflows[5,1]    arcflows[5,2]       arcflows[5,9]    arcflows[5,10]  
 arcflows[6,1]    arcflows[6,2]    …  arcflows[6,9]    arcflows[6,10]  
 arcflows[7,1]    arcflows[7,2]       arcflows[7,9]    arcflows[7,10]  
 arcflows[8,1]    arcflows[8,2]       arcflows[8,9]    arcflows[8,10]  
 arcflows[9,1]    arcflows[9,2]       arcflows[9,9]    arcflows[9,10]  
 arcflows[10,1]   arcflows[10,2]      arcflows[10,9]   arcflows[10,10] 
 arcflows[11,1]   arcflows[11,2]   …  arcflows[11,9]   arcflows[11,10] 
 arcflows[12,1]   arcflows[12,2]      arcflows[12,9]   arcflows[12,10] 
 arcflows[13,1]   arcflows[13,2]      arcflows[13,9]   arcflows[13,10] 
 ⋮                               

In [53]:
@variable(caldata, adms[1:n_type_1_nodes], Bin)

22-element Array{JuMP.Variable,1}:
 adms[1] 
 adms[2] 
 adms[3] 
 adms[4] 
 adms[5] 
 adms[6] 
 adms[7] 
 adms[8] 
 adms[9] 
 adms[10]
 adms[11]
 adms[12]
 adms[13]
 adms[14]
 adms[15]
 adms[16]
 adms[17]
 adms[18]
 adms[19]
 adms[20]
 adms[21]
 adms[22]

### Constraints

In [54]:
# • The total bi-directional flow on an arc (the sum of the two directions) can be no more than the
# bi-directional capacity.
for cycle in keys(cycles)
    for ring in 1:cycles[cycle]
        for i in 1:(length(cycle)-1)
            from_hub = cycle[i]
            to_hub = cycle[i+1]
            direction_1_key = arc_indices_to_descriptions[ind] = "hub_" * string(from_hub) * "_cycle_" * cycle * "_ring_" * 
                    string(ring) * "_to_hub_" * to_hub * "_cycle_" * cycle * "_ring_" * string(ring)
            direction_2_key = arc_indices_to_descriptions[ind] = "hub_" * to_hub * "_cycle_" * cycle * "_ring_" * string(ring) * 
                                    "_to_hub_" * from_hub * "_cycle_" * cycle * "_ring_" * string(ring)
            
#             println(arc_descriptions_to_indices[direction_1_key], ": ", arc_descriptions_to_indices[direction_2_key])
            @constraint(caldata, [i=1:length(arcflows)], 
                sum(arcflows[arc_descriptions_to_indices[direction_1_key],j]
                + arcflows[arc_descriptions_to_indices[direction_2_key],j] for j in 1:n_commodities)
                <= bidirectional_arc_capacity)
            
        end
        
        from_hub = cycle[1]
        to_hub = cycle[end]
        direction_1_key = arc_indices_to_descriptions[ind] = "hub_" * string(from_hub) * "_cycle_" * cycle * "_ring_" * 
                string(ring) * "_to_hub_" * to_hub * "_cycle_" * cycle * "_ring_" * string(ring)
        direction_2_key = arc_indices_to_descriptions[ind] = "hub_" * to_hub * "_cycle_" * cycle * "_ring_" * string(ring) * 
                                "_to_hub_" * from_hub * "_cycle_" * cycle * "_ring_" * string(ring)

        @constraint(caldata, [i=1:length(arcflows)], 
            sum(arcflows[arc_descriptions_to_indices[direction_1_key], j]
            + arcflows[arc_descriptions_to_indices[direction_2_key], j] for j in 1:n_commodities)
            <= bidirectional_arc_capacity)
    end
end

# No commodities may flow on a Type 0 arc unless their destination is at that hub.
for hub in type_0_nodes
    for cycle in hubs_to_cycles[hub]
        for ring in 1:cycles[cycle]
            # from ring to city (type 0)
            for j in 1:n_commodities
                if ! endswith(commodities[j], hub)
                    arc_key = "hub_" * hub * "_cycle_" * cycle * "_ring_" * string(ring) * "_to_hub_" * hub
                    @constraint(caldata, arcflows[arc_descriptions_to_indices[arc_key],j] == 0)
                end
            end
        end
    end
end

# • No commodities may flow on a Type 1 arc unless their origin is at that hub.
for hub in type_0_nodes
    for cycle in hubs_to_cycles[hub]
        for ring in 1:cycles[cycle]
            # from ring to city (type 0)
            node_descriptions_to_indices
            for j in 1:n_commodities
                if ! startswith(commodities[j], hub)
                    arc_key = "hub_" * hub * "_to_hub_" * hub * "_cycle_" * cycle * "_ring_" * string(ring)
                    @constraint(caldata, arcflows[arc_descriptions_to_indices[arc_key],j] == 0)
                end
            end
        end
    end
end


# No commodities may flow on a Type 0 (city to ring) arc unless an ADM is installed on that ring at that hub.
# No commodities may flow on a Type 1 (ring to city) arc unless an ADM is installed on that ring at that hub.
for hub in type_0_nodes
    for cycle in hubs_to_cycles[hub]
        for ring in 1:cycles[cycle]
            type_0_arc_key = "hub_" * hub * "_cycle_" * cycle * "_ring_" * string(ring) * "_to_hub_" * hub
            type_1_arc_key = "hub_" * hub * "_to_hub_" * hub * "_cycle_" * cycle * "_ring_" * string(ring)
            node_key = "hub_" * hub * "_cycle_" * cycle * "_ring_" * string(ring)
            for j in 1:n_commodities
                
                @constraint(caldata, arcflows[arc_descriptions_to_indices[type_0_arc_key],j]
                    <= adms[node_descriptions_to_indices[node_key]]*BIGM)
                
                @constraint(caldata, arcflows[arc_descriptions_to_indices[type_1_arc_key],j]
                    <= adms[node_descriptions_to_indices[node_key]]*BIGM)
                
            end
        end
    end
end

In [55]:
# No commodities may flow on a Type 3 arc unless an ADM is installed at that hub on both rings.
for hub in type_0_nodes
    if !hasbbdx(hub)
        continue
    end
    
    cycles_at_hub = hubs_to_cycles[hub]
    for cycle_num in 1:length(cycles_at_hub)
        # in-cycle transfers: across rings
        for pair in collect(combinations(collect(1:cycles[cycles_at_hub[cycle_num]]), 2))
            ring1, ring2 = pair
            arc_key_dir1 = "hub_"*hub*"_cycle_"*cycles_at_hub[cycle_num]*"_ring_"*string(ring1) *
                                                        "_to_hub_"*hub*"_cycle_"*cycles_at_hub[cycle_num]*"_ring_"*string(ring2)
            arc_key_dir2 = "hub_"*hub*"_cycle_"*cycles_at_hub[cycle_num]*"_ring_"*string(ring2) *
                                                        "_to_hub_"*hub*"_cycle_"*cycles_at_hub[cycle_num]*"_ring_"*string(ring1)
            
            ring_key_one = "hub_"*hub*"_cycle_"*cycles_at_hub[cycle_num]*"_ring_"*string(ring2)
            ring_key_two = "hub_"*hub*"_cycle_"*cycles_at_hub[cycle_num]*"_ring_"*string(ring1)
            
            for j in 1:n_commodities 
                @constraint(caldata, arcflows[arc_descriptions_to_indices[arc_key_dir1],j]
                        <= adms[node_descriptions_to_indices[ring_key_one]]*BIGM)
                @constraint(caldata, arcflows[arc_descriptions_to_indices[arc_key_dir1],j]
                        <= adms[node_descriptions_to_indices[ring_key_two]]*BIGM)
                
                
                @constraint(caldata, arcflows[arc_descriptions_to_indices[arc_key_dir2],j]
                        <= adms[node_descriptions_to_indices[ring_key_one]]*BIGM)
                @constraint(caldata, arcflows[arc_descriptions_to_indices[arc_key_dir2],j]
                        <= adms[node_descriptions_to_indices[ring_key_two]]*BIGM)

            end
        
        end

        # out-of-cycle transfers
        for other_cycle_num in append!(collect(1:cycle_num-1), collect(cycle_num+1:length(cycles_at_hub)))
            # for each ring in THIS cycle, make a link to the other ring in other cycles
            for ring in 1:cycles[cycles_at_hub[cycle_num]]
                for other_ring in 1:cycles[cycles_at_hub[other_cycle_num]]                        
                    ring_key_one = "hub_"*hub*"_cycle_"*cycles_at_hub[cycle_num]*"_ring_"*string(ring)
                    ring_key_two = "hub_"*hub*"_cycle_"* cycles_at_hub[other_cycle_num]*"_ring_"*string(other_ring)
                    arc_key = "hub_"*hub*"_cycle_"*cycles_at_hub[cycle_num]*"_ring_"*
                                                        string(ring) * "_to_hub_"*hub*"_cycle_"*
                                                        cycles_at_hub[other_cycle_num]*"_ring_"*string(other_ring)
                    for j in 1:n_commodities
                        @constraint(caldata, arcflows[arc_descriptions_to_indices[arc_key],j]
                                    <= adms[node_descriptions_to_indices[ring_key_one]]*BIGM)
                        
                        @constraint(caldata, arcflows[arc_descriptions_to_indices[arc_key],j]
                                    <= adms[node_descriptions_to_indices[ring_key_two]]*BIGM)
                    end
                end
            end
        end
    end
end

In [56]:
# The total flow of each commodity into its destination hub (i.e., into a Type 0 node on Type 0
# arcs) must equal the demand for that commodity.
for hub in type_0_nodes
    for j in 1:n_commodities
        # if the hub is the destination of the jth commodity
        if endswith(commodities[j], hub)
            # get the indices of the 0-arcs going into hub from all rings, from all cycles
            arc_inds = []
            for c in hubs_to_cycles[string(hub)]
                for ring in 1:cycles[c]
                    arc_desc = "hub_"*hub*"_cycle_"*c*"_ring_"*string(ring)*"_to_hub_"*hub
                    arc_ind = arc_descriptions_to_indices[arc_desc]
                    push!(arc_inds, arc_ind)
                end
            end
            # the sum of all those arcs, for the jth commodity, should be equal to the demand of that commodity
            @constraint(caldata, sum(arcflows[arc_ind, j] for arc_ind in arc_inds) == demands[commodities[j]])
        end
    end
end

In [57]:
# • The total flow of each commodity out of its origin hub (i.e., out of a Type 0 node on Type 1
# arcs) must equal the demand for that commodity.
for hub in type_0_nodes
    for j in 1:n_commodities
        # if the hub is the ORIGIN of the jth commodity
        if startswith(commodities[j], hub)
            # get the indices of the 0-arcs going into hub from all rings, from all cycles
            arc_inds = []
            for c in hubs_to_cycles[string(hub)]
                for ring in 1:cycles[c]
                    arc_desc = "hub_" * hub * "_to_hub_" * hub * "_cycle_" * c * "_ring_" * string(ring)                    
                    arc_ind = arc_descriptions_to_indices[arc_desc]
                    push!(arc_inds, arc_ind)
                end
            end
            # the sum of all those arcs, for the jth commodity, should be equal to the demand of that commodity
            @constraint(caldata, sum(arcflows[arc_ind, j] for arc_ind in arc_inds) == demands[commodities[j]])
        end
    end
end

In [58]:
# push!([[1 2 3]], [1 2 3])

In [59]:
# The total flow of each commodity into each Type 1 node must equal the total flow of that
# commodity out of that Type 1 node
inds_for_ring_switching_arcs = []
inds_for_entering_or_leaving_network = []

for (hub_i, hub) in collect(enumerate(type_0_nodes))
    cycles_at_hub = hubs_to_cycles[hub]
    for (cycle_i, cycle) in collect(enumerate(cycles_at_hub))
        for ring in 1:cycles[cycle]
            arc_inds_in, arc_inds_out = [], []
            # get everything into it (type 1, 2, and 3 arc)
            
            # IN: type 1 arc (city to ring)
            arc = "hub_" * hub * "_to_hub_" * hub * "_cycle_" * cycle * "_ring_" * string(ring)
            push!(arc_inds_in, arc_descriptions_to_indices[arc])  
            
            push!(inds_for_entering_or_leaving_network, arc_descriptions_to_indices[arc])
            
            # OUT: type 0 arc (ring to city)
            arc = "hub_" * hub * "_cycle_" * cycle *"_ring_" * string(ring) * "_to_hub_" * hub
            push!(arc_inds_out, arc_descriptions_to_indices[arc]) 
            
            push!(inds_for_entering_or_leaving_network, arc_descriptions_to_indices[arc])

            
            # type 2 arcs (from an adjacent hub, but on the same ring)
            hub_index_in_cycle = search(cycle, hub)[1]
            if hub == string(cycle[1]) ### C
                neighboring_hub_left = cycle[end]
            else
                neighboring_hub_left = cycle[hub_index_in_cycle - 1]
            end

            
            if hub == string(cycle[end])
                neighboring_hub_right = cycle[1]
            else
                neighboring_hub_right = cycle[hub_index_in_cycle + 1]
            end
            
            
            # IN: type 2 arc, neighbor left
            arc = "hub_" * neighboring_hub_left * "_cycle_" * cycle * "_ring_" * 
                    string(ring) * "_to_hub_" * hub * "_cycle_" * cycle *
                    "_ring_" * string(ring)
            
            push!(arc_inds_in, arc_descriptions_to_indices[arc]) 

            # OUT: type 2 arc, neighbor left
            arc = "hub_" * hub * "_cycle_" * cycle * "_ring_" * string(ring) * "_to_hub_" * neighboring_hub_left * "_cycle_" * cycle*
                    "_ring_" * string(ring)
            push!(arc_inds_out, arc_descriptions_to_indices[arc])
            
            # IN: type 2 arc, neighbor right
            arc = "hub_" * neighboring_hub_right * "_cycle_" * cycle * "_ring_" * 
                    string(ring) * "_to_hub_" * hub * "_cycle_" * cycle *
                    "_ring_" * string(ring)
            push!(arc_inds_in, arc_descriptions_to_indices[arc])
            
            # OUT: type 2 arc, neighbor right
            arc = "hub_" * hub * "_cycle_" * cycle * "_ring_" * 
                                string(ring) * "_to_hub_" * neighboring_hub_right * "_cycle_" * cycle *
                                "_ring_" * string(ring)
            push!(arc_inds_out, arc_descriptions_to_indices[arc])
            

            # type 3 arcs (switching rings) - only if BBDX installed
            if hasbbdx(hub)
                for other_ring_same_cycle in 1:cycles[cycle]
                    if ring == other_ring_same_cycle
                        continue
                    end
                    # IN: in-cycle in-transfers: across rings
                    arc = "hub_"*hub*"_cycle_"*cycle*"_ring_"*string(other_ring_same_cycle) *
                                                                "_to_hub_"*hub*"_cycle_"*cycle*
                                                        "_ring_"*string(ring)
                    push!(arc_inds_in, arc_descriptions_to_indices[arc])
                    push!(inds_for_ring_switching_arcs, arc_descriptions_to_indices[arc])

                    # OUT: in-cycle in-transfers: from this ring to other ones
                    arc = "hub_"*hub*"_cycle_"*cycle*"_ring_"*string(ring) *
                                                                "_to_hub_"*hub*"_cycle_"*cycle*
                                                        "_ring_"*string(other_ring_same_cycle)
                    push!(arc_inds_out, arc_descriptions_to_indices[arc])
                    push!(inds_for_ring_switching_arcs, arc_descriptions_to_indices[arc])

                end

                # out-of-cycle in-transfers
                for other_cycle_num in append!(collect(1:cycle_i-1), collect(cycle_i+1:length(cycles_at_hub)))
                    for other_ring in 1:cycles[cycles_at_hub[other_cycle_num]]
                        # IN: out-of-cycle in-transfers: across rings
                        arc = "hub_"*hub*"_cycle_"*cycles_at_hub[other_cycle_num]*"_ring_"*
                                                            string(other_ring) * "_to_hub_"*hub*"_cycle_"*
                                                            cycle*"_ring_"*string(ring)
                        push!(arc_inds_in, arc_descriptions_to_indices[arc])
                        push!(inds_for_ring_switching_arcs, arc_descriptions_to_indices[arc])

                        # OUT: out-of-cycle in-transfers: from this ring to other ones
                        arc = "hub_"*hub*"_cycle_"*cycle*"_ring_"*
                                                            string(ring) * "_to_hub_"*hub*"_cycle_"*
                                                            cycles_at_hub[other_cycle_num]*"_ring_"*string(other_ring)

                        push!(arc_inds_out, arc_descriptions_to_indices[arc])
                        push!(inds_for_ring_switching_arcs, arc_descriptions_to_indices[arc])
                    end
                end

                @constraint(caldata,[j=1:n_commodities], sum(arcflows[i,j] for i in arc_inds_in)
                                                        == sum(arcflows[k,j] for k in arc_inds_out))
            end
        end
    end
end

In [60]:
all_adm_transfer_inds = vcat(inds_for_ring_switching_arcs, inds_for_entering_or_leaving_network);

In [61]:
# total flow cost for a commodity
arcflow_type_2_indices = []
unit_cost_arcflow_factors = []
# type 2 arcs: one hub to the next along a ring (bidirectional, not symmetric)
for cycle in keys(cycles)
    for ring in 1:cycles[cycle]
        for i in 1:(length(cycle)-1)
            from_hub = cycle[i]
            to_hub = cycle[i+1]
            arc = "hub_" * string(from_hub) * "_cycle_" * cycle * "_ring_" * 
                    string(ring) * "_to_hub_" * to_hub * "_cycle_" * cycle * "_ring_" * string(ring)
            
            push!(arcflow_type_2_indices, arc_descriptions_to_indices[arc])
            push!(unit_cost_arcflow_factors, flowcost(from_hub*to_hub))

            
             arc = "hub_" * to_hub * "_cycle_" * cycle * "_ring_" * string(ring) * 
                                    "_to_hub_" * from_hub * "_cycle_" * cycle * "_ring_" * string(ring)

            push!(arcflow_type_2_indices, arc_descriptions_to_indices[arc])
            push!(unit_cost_arcflow_factors, flowcost(from_hub*to_hub))
        
        end
        arc = "hub_" * cycle[1] * "_cycle_" * cycle * "_ring_" * string(ring) * 
                            "_to_hub_" * cycle[end] * "_cycle_" * cycle * "_ring_" * string(ring)

        push!(arcflow_type_2_indices, arc_descriptions_to_indices[arc])
        push!(unit_cost_arcflow_factors, flowcost(cycle[1]*cycle[end]))
        
        arc = "hub_" * cycle[end] * "_cycle_" * cycle * "_ring_" * string(ring) * 
                        "_to_hub_" * cycle[1] * "_cycle_" * cycle * "_ring_" * string(ring)

        push!(arcflow_type_2_indices, arc_descriptions_to_indices[arc])
        push!(unit_cost_arcflow_factors, flowcost(cycle[end]*cycle[1]))
    
    end
end

num_arcs_type_2 = length(arcflow_type_2_indices)

44

In [62]:
# arcflow_type_2_indices

In [63]:
flowcost(commodities[8])

0

In [64]:
# the price of installing ADMs plus all the data that flows over them
@objective(caldata, Min, 1e6*sum(adms[i] for i in 1:n_type_1_nodes) +
                        2*sum(arcflows[i,j] for j in 1:n_commodities for i in all_adm_transfer_inds) +
                sum(arcflows[arcflow_type_2_indices[i],j]*unit_cost_arcflow_factors[i] for j in 1:n_commodities 
                    for i in 1:num_arcs_type_2));


In [65]:
solve(caldata)

Academic license - for non-commercial use only
Optimize a model with 38114 rows, 1642 columns and 719640 nonzeros
Variable types: 1620 continuous, 22 integer (22 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 4e+04]
Presolve removed 36345 rows and 374 columns
Presolve time: 0.27s
Presolved: 1769 rows, 1268 columns, 5928 nonzeros
Variable types: 1250 continuous, 18 integer (18 binary)

Root relaxation: objective 7.353617e+06, 372 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7353616.85    0   18          - 7353616.85      -     -    0s
H    0     0                    1.777400e+07 7353616.85  58.6%     -    0s
     0     0 7655535.92    0   18 1.7774e+07 7655535.92  56.9%     -    0s
     0     0 7992724.36    0   15 1.7774e+07 799272

:Optimal

In [66]:
af = getvalue(arcflows)
adm_s = getvalue(adms)

22-element Array{Float64,1}:
  1.0
  1.0
  1.0
  0.0
  1.0
  1.0
  1.0
 -0.0
  0.0
  1.0
  0.0
 -0.0
 -0.0
 -0.0
  1.0
 -0.0
  1.0
  0.0
  1.0
  0.0
 -0.0
 -0.0

In [68]:
for (i,adm) in collect(enumerate(adm_s))
    if adm > 0.0001
        println(node_indices_to_descriptions[i])
    end
end



# for i in 1:size(af)[1]
#     arc = af[i,:]
#     for j in 1:n_commodities
#         if arc[j] > 0
#             desc = arc_indices_to_descriptions[i]
#             if contains(desc, "hub_A_cycle_ABEC")
#                 println(desc, ": ", string(j))
#             end
#         end
#     end
# end

hub_A_cycle_ABEC_ring_1
hub_A_cycle_ABEC_ring_2
hub_A_cycle_ABEC_ring_3
hub_B_cycle_ABEC_ring_1
hub_B_cycle_ABEC_ring_2
hub_B_cycle_ABEC_ring_3
hub_E_cycle_ABEC_ring_2
hub_C_cycle_ABEC_ring_3
hub_B_cycle_BDE_ring_1
hub_D_cycle_BDE_ring_1


nothing is switching rings

## TODO
1. neaten code (trim and modularize)
2. run on cluster
3. run on caldata surface level

In [ ]:
has_bbdx = Dict("A" => true,
        "B" => true,
        "C" => false,
        "D" => true,
        "E" => true,
        "F" => false,
        "G" => false,
        "H" => true,
        "I" => false,
        "J" => true,
        "K" => true,
        "L" => true,
        "M" => false,
        "N" => true,
);

In [ ]:
# for k in keys(arc_indices_to_descriptions)
#     val = arc_indices_to_descriptions[k]
#     if startswith(val, "hub_N_cycle_KLNM") #&& ! contains(val, "K")  && ! contains(val, "H")
#         println(val)
#     end
# end